# Register Layouts & Mappable Registers

In [ ]:
import numpy as np

from pulser.register.register_layout import RegisterLayout
from pulser import Sequence, Pulse

One of the strengths of neutral-atom QPUs is their ability to arrange the atoms in arbitrary configurations. Experimentally, this is realized by creating a layout of optical traps where individual atoms can be placed to create the desired Register. 

Given an arbitrary register, a neutral-atom QPU will generate an associated layout that will then have to be calibrated. Each new calibration takes some time, so it is often prefered to reuse an existing layout that has already been calibrated, whenever possible.

Therefore, it can be of interest to the QPU provider to specify which layouts are already calibrated in their QPU, such that the user can reuse them to specify their `Register`. In Pulser, these layouts are provided as instances of the `RegisterLayout` class.

## Arbitrary Layouts

A `RegisterLayout` layout is defined by a set of trap coordinates. These coordinates are systematically ordered in the same way, making two layouts with the same set of trap coordinates identical. 

Below, we create an arbitrary layout of 20 traps randomly placed in a 2D plane. Optionally, a layout may also have an associated `slug` to help identifying it.

In [ ]:
# Generating random coordinates
np.random.seed(301122)  # Keeps results consistent between runs
traps = np.random.randint(0, 30, size=(20, 2))
traps = traps - np.mean(traps, axis=0)

# Creating the layout
layout = RegisterLayout(traps, slug="random_20")

Given a `RegisterLayout` instance, the best way to inspect it is through `RegisterLayout.draw()`. Notice the default ordering of the atoms (ascending order in x, if tied then in y, if tied then in z):

In [ ]:
layout.draw()

### Useful properties

To access the trap coordinates:
- `RegisterLayout.traps_dict` gives a mapping between trap IDs and coordinates
- `RegisterLayout.coords` provides the ordered list of trap coordinates

To identify a layout, one can use its `repr()` for a unique identifier or its `str()` for the `slug` (if specified).

In [ ]:
print("The layout slug:", layout)
print("The unique ID layout:", repr(layout))

Finally, `RegisterLayout.max_atom_num` fixes the maximum number of atoms it can hold (for now, this value is always equal to half the number of traps):

In [ ]:
print("Maximum number of atoms supported:", layout.max_atom_num)

### Register definition

More often than not, a `RegisterLayout` will be created by the hardware provider and given to the user. From there, the user must define the desired `Register` to initialize the `Sequence`. This can be done in multiple ways: 

**1. Defined by the trap IDs:**

You can find the ID of each trap from its drawing or from the `RegisterLayout.traps_dict`. With those, you can define your register (optionally providing a list of qubit IDs):

In [ ]:
trap_ids = [4, 8, 19, 0]
reg1 = layout.define_register(*trap_ids, qubit_ids=["a", "b", "c", "d"])
reg1.draw()

Note that the information of the layout is stored internally in the Register:

In [ ]:
reg1.layout

**2. Defined from the trap coordinates:**

Alternatively, you can find the trap IDs from the trap coordinates using the `RegisterLayout.get_traps_from_coordinates()` method, which compares the provided coordinates with those on the layout with 6 decimal places of precision.

In [ ]:
some_coords = layout.coords[
    np.random.choice(np.arange(layout.number_of_traps), size=10, replace=False)
]
trap_ids = layout.get_traps_from_coordinates(*some_coords)
reg2 = layout.define_register(*trap_ids)
reg2.draw()

## Special Layouts

In [ ]:
from pulser.register.special_layouts import (
    SquareLatticeLayout,
    TriangularLatticeLayout,
)

On top of the generic `RegisterLayout` class, there are special classes for common layouts that include convenience methods to more easily define a `Register`. These are subclasses of `RegisterLayout`, so all the methods specified above will still work.

### `SquareLatticeLayout`

`SquareLatticeLayout` specifies a layout from an underlying square lattice.

In [ ]:
square_layout = SquareLatticeLayout(7, 4, spacing=5)
print(square_layout)
square_layout.draw()

With `SquareLatticeLayout.rectangular_register()` and `SquareLatticeLayout.square_register()`, one can conveniently define a new `Register`:

In [ ]:
square_layout.rectangular_register(rows=3, columns=4, prefix="a").draw()

In [ ]:
square_layout.square_register(side=3).draw()

### `TriangularLatticeLayout`

`TriangularLatticeLayout` specifies a layout from an underlying triangular lattice.

In [ ]:
tri_layout = TriangularLatticeLayout(n_traps=100, spacing=5)
print(tri_layout)
tri_layout.draw()

With `TriangularLatticeLayout.hexagonal_register()` or `TriangularLatticeLayout.rectangular_register()`, one can easily define a `Register` from a subset of existing traps.

In [ ]:
tri_layout.hexagonal_register(n_atoms=50).draw()

In [ ]:
tri_layout.rectangular_register(rows=3, atoms_per_row=7).draw()

## Devices with pre-calibrated layouts

In [ ]:
from pulser.devices import Device
from pulser.channels import Rydberg, Raman

TestDevice = Device(
    name="TestDevice",
    dimensions=2,
    rydberg_level=70,
    max_atom_num=100,
    max_radial_distance=50,
    min_atom_distance=4,
    _channels=(
        ("rydberg_global", Rydberg.Global(2 * np.pi * 20, 2 * np.pi * 2.5)),
    ),
    pre_calibrated_layouts=(
        SquareLatticeLayout(10, 10, 4),
        TriangularLatticeLayout(100, 5),
    ),
)

When receiving a `Device` instance, it may include the layouts that are already calibrated and available to be used. To access them, simply run:

In [ ]:
TestDevice.calibrated_register_layouts

You can then choose one of these layouts to define your `Register` and start creating a `Sequence`:

In [ ]:
layout = TestDevice.calibrated_register_layouts[
    "SquareLatticeLayout(10x10, 4µm)"
]
reg = layout.square_register(7)
seq = Sequence(reg, TestDevice)

In general, when a device comes with `pre_calibrated_layouts`, using them is encouraged. However, nothing prevents a `Sequence` to be created with a register coming from another layout, as long as that layout is compatible with the device. For example:

In [ ]:
another_layout = SquareLatticeLayout(5, 5, 5)
assert another_layout not in TestDevice.pre_calibrated_layouts
reg_ = another_layout.square_register(3)
seq = Sequence(reg_, TestDevice)

However, it is not possible to use a register created from an invalid layout, even if the register is valid:

In [ ]:
bad_layout = TriangularLatticeLayout(
    200, 10
)  # This layout is too large for TestDevice
good_reg = bad_layout.hexagonal_register(
    10
)  # On its own, this register is valid in TestDevice
try:
    seq = Sequence(good_reg, TestDevice)
except ValueError as e:
    print(e)

## Mappable Registers

Finally, layouts enable the creation of a `MappableRegister` — a register with the traps of each qubit still to be defined. This register can then be used to create a sort of parametrized `Sequence`, where deciding which traps will be mapped to which qubits only happens when `Sequence.build()` is called.

For example, below we define a mappable register with 10 qubits.

In [ ]:
map_register = layout.make_mappable_register(n_qubits=10)
map_register.qubit_ids

We now use this register in our simple sequence:

In [ ]:
seq = Sequence(map_register, TestDevice)
assert seq.is_register_mappable()

seq.declare_channel("rydberg", "rydberg_global")
seq.add(
    Pulse.ConstantPulse(duration=100, amplitude=1, detuning=0, phase=0),
    "rydberg",
)
seq.draw()

To define the register, we can then call `Sequence.build()`, indicated in the `qubits` argument the map between qubit IDs and trap IDs (note that not all the qubit IDs need to be associated to a trap ID). 

In this way, we can build multiple sequences, with only the `Register` changing from one to the other:

In [ ]:
seq1 = seq.build(qubits={"q0": 16, "q1": 19, "q4": 34})
print("First register:", seq1.register.qubits)

seq2 = seq.build(qubits={"q0": 0, "q1": 15, "q2": 20})
print("Second register:", seq2.register.qubits)